# Dynamically Returning Directly

A typical ReAct loop follows user -> assistant -> tool -> assistant ..., -> user. In some cases, you don't need to call the LLM after the tool completes, the user can view the results directly themselves. 

In this example we will build a conversational ReAct agent where the LLM can optionally decide to return the result of a tool call as the final answer. This is useful in cases where you have tools that can sometimes generate responses that are acceptable as final answers, and you want to use the LLM to determine when that is the case

## Setup

First we need to install the packages required

```bash
yarn add langchain @langchain/openai @langchain/langgraph
```

In [ ]:
import "dotenv/config";

Next, we need to set API keys for OpenAI (the LLM we will use) and Tavily (the search tool we will use)

```bash
export OPENAI_API_KEY=
export TAVILY_API_KEY=
```

Optionally, we can set API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.

```bash
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=
```

## Set up the tools

We will first define the tools we want to use.
For this simple example, we will use a built-in search tool via Tavily.
However, it is really easy to create your own tools - see documentation [here](https://js.langchain.com/docs/modules/agents/tools/dynamic) on how to do that.

In [ ]:
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";
import { z } from "zod";

const SearchTool = z.object({
  query: z.string().describe("query to look up online"),
  return_direct: z.boolean().describe("Whether or the result of this should be returned directly to the user without you seeing what it is").default(false),
});

const search_tool = new TavilySearchResults({ maxResults: 1, schema: SearchTool });
const tools = [search_tool];

We can now wrap these tools in a simple ToolExecutor.  
This is a real simple class that takes in a ToolInvocation and calls that tool, returning the output.
A ToolInvocation is any type with `tool` and `toolInput` attribute.

In [ ]:
import { ToolExecutor } from "@langchain/langgraph/prebuilt";

const toolExecutor = new ToolExecutor({
  tools,
});

## Set up the model

Now we need to load the chat model we want to use.  
Importantly, this should satisfy two criteria:

1. It should work with messages. We will represent all agent state in the form of messages, so it needs to be able to work well with them.
2. It should work with OpenAI function calling. This means it should either be an OpenAI model or a model that exposes a similar interface.

Note: these model requirements are not requirements for using LangGraph - they are just requirements for this one example.

In [ ]:
import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI({ temperature: 0 });

After we've done this, we should make sure the model knows that it has these tools available to call.
We can do this by converting the LangChain tools into the format for OpenAI function calling, and then bind them to the model class.

In [ ]:
const boundModel = model.bindTools(tools);

## Define the agent state

The main type of graph in `langgraph` is the [StateGraph](https://langchain-ai.github.io/langgraphjs/reference/classes/index.StateGraph.html).
 
This graph is parameterized by a state object that it passes around to each node.
Each node then returns operations to update that state. 
These operations can either SET specific attributes on the state (e.g. overwrite the existing values) or ADD to the existing attribute.
Whether to set or add is denoted in the state object you construct the graph with.

For this example, the state we will track will just be a list of messages.
We want each node to just add messages to that list.
Therefore, we will define the state as follows:

In [ ]:
interface AgentStateBase {
  messages: Array<BaseMessage>;
}

interface AgentState extends AgentStateBase {}

const agentState = {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => x.concat(y),
    default: () => [],
  },
};

## Define the nodes

We now need to define a few different nodes in our graph. 
In `langgraph`, a node can be either a function or a [runnable](https://js.langchain.com/docs/expression_language/).
There are two main nodes we need for this:

1. The agent: responsible for deciding what (if any) actions to take.
2. A function to invoke tools: if the agent decides to take an action, this node will then execute that action.

We will also need to define some edges.
Some of these edges may be conditional.
The reason they are conditional is that based on the output of a node, one of several paths may be taken. 
The path that is taken is not known until that node is run (the LLM decides).

1. Conditional Edge: after the agent is called, we should either:
   a. If the agent said to take an action, then the function to invoke tools should be called
   b. If the agent said that it was finished, then it should finish  
2. Normal Edge: after the tools are invoked, it should always go back to the agent to decide what to do next

Let's define the nodes, as well as a function to decide how what conditional edge to take.

In [ ]:
import { BaseMessage } from "@langchain/core/messages";
import { AgentAction } from "@langchain/core/agents";
import type { RunnableConfig } from "@langchain/core/runnables";

// Define the function that determines whether to continue or not 
const shouldContinue = (state: AgentState) => {
  const messages = state.messages;
  const lastMessage = messages[messages.length - 1];
  // If there is no function call, then we finish
  if (!lastMessage.additional_kwargs.function_call) {
    return "end";
  }
  // Otherwise if there is, we check if it's suppose to return direct  
  else {
    const arguments = JSON.parse(lastMessage.additional_kwargs.function_call.arguments); 
    if (arguments.return_direct) {
      return "final";
    } else {
      return "continue";      
    }
  }
};

// Define the function that calls the model
const callModel = async (state: AgentState, config: RunnableConfig) => {
  const messages = state.messages;
  const response = await boundModel.invoke(messages, config);
  // We return an object, because this will get added to the existing list
  return { messages: [response] };
};

// Define the function to execute tools  
const callTool = async (state: AgentState, config: RunnableConfig) => {
  const messages = state.messages;
  // Based on the continue condition 
  // we know the last message involves a function call
  const lastMessage = messages[messages.length - 1];
  // We construct an AgentAction from the function_call
  const toolCall = lastMessage.additional_kwargs.function_call;
  const toolName = toolCall.name;
  let arguments = JSON.parse(toolCall.arguments);
  if (toolName === "tavily_search_results") {
    if ("return_direct" in arguments) {
      delete arguments["return_direct"];
    }
  }
  const action: AgentAction = {
    tool: toolName,
    toolInput: JSON.stringify(arguments),
    log: "",
  };
  // We call the tool_executor and get back a response 
  const response = await toolExecutor.invoke(action, config);
  // We use the response to create a ToolMessage
  const toolMessage = new ToolMessage({  
    content: response,
    name: action.tool,
    tool_call_id: toolCall.id,
  });
  // We return an object, because this will get added to the existing list
  return { messages: [toolMessage] };
};

## Define the graph

We can now put it all together and define the graph!

In [ ]:
import { END, StateGraph } from "@langchain/langgraph";

// Define a new graph  
const workflow = new StateGraph({
  channels: agentState,
});

// Define the two nodes we will cycle between
workflow.addNode("agent", callModel);
  
// Note the "action" and "final" nodes are identical!
workflow.addNode("action", callTool);  
workflow.addNode("final", callTool);

// Set the entrypoint as `agent`
workflow.setEntryPoint("agent");  

// We now add a conditional edge
workflow.addConditionalEdges(
  // First, we define the start node. We use `agent`. 
  "agent",
  // Next, we pass in the function that will determine which node is called next.
  shouldContinue,
  // Finally we pass in a mapping. 
  {
    // If `tools`, then we call the tool node.
    continue: "action",
    // Final call 
    final: "final",  
    // Otherwise we finish.
    end: END,
  },
);

// We now add a normal edge from `tools` to `agent`.
workflow.addEdge("action", "agent");
workflow.addEdge("final", END);  

// Finally, we compile it!
const app = workflow.compile();

## Use it!

We can now use it!
This now exposes the [same interface](https://js.langchain.com/docs/expression_language/) as all other LangChain runnables.

In [ ]:
import { HumanMessage } from "@langchain/core/messages";

const inputs = { messages: [new HumanMessage("what is the weather in sf")] };  
for await (const output of await app.stream(inputs)) {
  console.log(output);
  console.log("-----\n");
}

In [ ]:
import { HumanMessage } from "@langchain/core/messages";

const inputs = {
  messages: [
    new HumanMessage(
      "what is the weather in sf? return this result directly by setting return_direct = True",  
    ),
  ],
};
for await (const output of await app.stream(inputs)) {
  console.log(output);
  console.log("\n---\n");
}